In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
# Download NLTK stop words
nltk.download('stopwords')
nltk.download('punkt')

from google.colab import drive
drive.mount('/content/drive')

import os
import json
from tqdm import tqdm  # For progress bar
import time  # For timing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/MyDrive/compliance/OPP-115/annotations/'

In [ ]:
column_headers = ['doc_id', 'test_category', 'segment_number', 'unique_policy_identifier', 'label', 'policy_category', 'segment_details', 'policy_date', 'url']

def read_all_policy_files_with_headers(folder_path):
    all_policies = []
    file_count = 0

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            file_count += 1
            print(f"Reading file {file_count}: {file_name}")
            df = pd.read_csv(file_path, header=None)
            df.columns = column_headers
            all_policies.append(df)

    print(f"Total CSV files read: {file_count}")

    combined_df = pd.concat(all_policies, ignore_index=True)

    return combined_df

combined_df = read_all_policy_files_with_headers(folder_path)

print(combined_df.shape)
print(combined_df.columns)

Reading file 1: 744_minecraft.gamepedia.com.csv
Reading file 2: 883_ted.com.csv
Reading file 3: 898_uptodate.com.csv
Reading file 4: 164_adweek.com.csv
Reading file 5: 175_mlb.mlb.com.csv
Reading file 6: 1164_acbj.com.csv
Reading file 7: 1666_wsmv.com.csv
Reading file 8: 1713_latinpost.com.csv
Reading file 9: 1264_citizen.org.csv
Reading file 10: 856_sciencemag.org.csv
Reading file 11: 186_abcnews.com.csv
Reading file 12: 200_washingtonpost.com.csv
Reading file 13: 541_ifsa-butler.org.csv
Reading file 14: 1708_foxsports.com.csv
Reading file 15: 1419_miaminewtimes.com.csv
Reading file 16: 26_nytimes.com.csv
Reading file 17: 559_www.loc.gov.csv
Reading file 18: 1694_lids.com.csv
Reading file 19: 686_military.com.csv
Reading file 20: 1498_ticketmaster.com.csv
Reading file 21: 414_washingtonian.com.csv
Reading file 22: 586_cariboucoffee.com.csv
Reading file 23: 591_google.com.csv
Reading file 24: 1089_freep.com.csv
Reading file 25: 32_voxmedia.com.csv
Reading file 26: 919_uh.edu.csv
Readin

### 1. EDA

In [ ]:
category_count = combined_df['policy_category'].value_counts()

fig = px.bar(category_count,
             x=category_count.index,
             y=category_count.values,
             labels={'x': 'Policy Category', 'y': 'Count'},
             title='Distribution of Policy Categories',
             width=800,
             height=600)

fig.update_layout(title={'text': 'Distribution of Policy Categories', 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},xaxis_title='Policy Category' )

fig.show()


In [ ]:
combined_df.head(2)

,doc_id,test_category,segment_number,unique_policy_identifier,label,policy_category,segment_details,policy_date,url
0,4889,test_category_labeling_highlight,84,3721,0,Other,"{""Other Type"": {""endIndexInSegment"": 1066, ""st...",10/2/12,http://minecraft.gamepedia.com/Minecraft_Wiki:...
1,5932,test_category_labeling_highlight,88,3721,0,Other,"{""Other Type"": {""endIndexInSegment"": 1066, ""st...",10/2/12,http://minecraft.gamepedia.com/Minecraft_Wiki:...


In [ ]:
# Select the row of interest (for example, row 2)
row_data = combined_df.iloc[2]

# Extract values for other columns along with segment_details
doc_id = row_data['doc_id']
test_category = row_data['test_category']
segment_number = row_data['segment_number']
unique_policy_identifier = row_data['unique_policy_identifier']
label = row_data['label']
policy_category = row_data['policy_category']
policy_date = row_data['policy_date']
url = row_data['url']

# Convert segment_details from JSON string to a dictionary
segment_details_sample = row_data['segment_details']
segment_details_dict = json.loads(segment_details_sample)

# Pretty print the dictionary and other columns
print(f"Doc ID: {doc_id}")
print(f"Test Category: {test_category}")
print(f"Segment Number: {segment_number}")
print(f"Unique Policy Identifier: {unique_policy_identifier}")
print(f"Label: {label}")
print(f"Policy Category: {policy_category}")
print(f"Policy Date: {policy_date}")
print(f"URL: {url}")
print("Segment Details:")
print(json.dumps(segment_details_dict, indent=4))


Doc ID: 5933
Test Category: test_category_labeling_highlight
Segment Number: 88
Unique Policy Identifier: 3721
Label: 0
Policy Category: Policy Change
Policy Date: 10/2/12
URL: http://minecraft.gamepedia.com/Minecraft_Wiki:Privacy_policy
Segment Details:
{
    "Change Type": {
        "endIndexInSegment": 816,
        "startIndexInSegment": 617,
        "selectedText": "By visiting the Site(s) or using any of the Curse Features, you consent to Curse' collection, use, storage, deletion and disclosure of information relating to you as set forth in this Privacy Policy.",
        "value": "Unspecified"
    },
    "User Choice": {
        "endIndexInSegment": 816,
        "startIndexInSegment": 617,
        "selectedText": "By visiting the Site(s) or using any of the Curse Features, you consent to Curse' collection, use, storage, deletion and disclosure of information relating to you as set forth in this Privacy Policy.",
        "value": "User participation"
    },
    "Notification Type":

### 2. Create a new column to store the policy chunk

In [ ]:
import json

def extract_policy_chunk(segment_details_sample):
    try:
        segment_details_dict = json.loads(segment_details_sample)

        policy_chunk = ""
        for key, value in segment_details_dict.items():
            policy_chunk += value['selectedText'] + " "

        return policy_chunk.strip()

    except Exception as e:
        return ""
combined_df['policy_chunk'] = combined_df['segment_details'].apply(extract_policy_chunk)

combined_df[['doc_id', 'url', 'policy_chunk']].head()


,doc_id,url,policy_chunk
0,4889,http://minecraft.gamepedia.com/Minecraft_Wiki:...,"Welcome to Curse! Curse, Inc. & Curse, Ltd. (""..."
1,5932,http://minecraft.gamepedia.com/Minecraft_Wiki:...,This Privacy Policy is effective as of the dat...
2,5933,http://minecraft.gamepedia.com/Minecraft_Wiki:...,By visiting the Site(s) or using any of the Cu...
3,6690,http://minecraft.gamepedia.com/Minecraft_Wiki:...,"Welcome to Curse! Curse, Inc. & Curse, Ltd. (""..."
4,4890,http://minecraft.gamepedia.com/Minecraft_Wiki:...,


### 2. Identify duplicates within a company url

In [ ]:
duplicates = combined_df[combined_df.duplicated(subset=['url', 'policy_chunk'], keep=False)]

num_duplicates = duplicates.shape[0]

print(f"Number of duplicates found: {num_duplicates}")
print(duplicates[['doc_id', 'url', 'policy_chunk']].head())


Number of duplicates found: 13674
   doc_id                                                url  \
0    4889  http://minecraft.gamepedia.com/Minecraft_Wiki:...   
3    6690  http://minecraft.gamepedia.com/Minecraft_Wiki:...   
4    4890  http://minecraft.gamepedia.com/Minecraft_Wiki:...   
5    5934  http://minecraft.gamepedia.com/Minecraft_Wiki:...   
6    6705  http://minecraft.gamepedia.com/Minecraft_Wiki:...   

                                        policy_chunk  
0  Welcome to Curse! Curse, Inc. & Curse, Ltd. ("...  
3  Welcome to Curse! Curse, Inc. & Curse, Ltd. ("...  
4                                                     
5                                                     
6                                                     


### 3. Drop those duplicates

In [ ]:
combined_df_cleaned = combined_df.drop_duplicates(subset=['url', 'policy_chunk'])

print(f"Number of duplicates after cleaning: {combined_df_cleaned.duplicated(subset=['url', 'policy_chunk']).sum()}")
print("Cleaned DataFrame:")
print(combined_df_cleaned.shape)
combined_df_cleaned[['doc_id', 'url', 'policy_chunk']].head()


Number of duplicates after cleaning: 0
Cleaned DataFrame:
(10897, 10)


,doc_id,url,policy_chunk
0,4889,http://minecraft.gamepedia.com/Minecraft_Wiki:...,"Welcome to Curse! Curse, Inc. & Curse, Ltd. (""..."
1,5932,http://minecraft.gamepedia.com/Minecraft_Wiki:...,This Privacy Policy is effective as of the dat...
2,5933,http://minecraft.gamepedia.com/Minecraft_Wiki:...,By visiting the Site(s) or using any of the Cu...
4,4890,http://minecraft.gamepedia.com/Minecraft_Wiki:...,
60,4911,http://minecraft.gamepedia.com/Minecraft_Wiki:...,We do not knowingly collect personal informati...


### 4. Check for na values in policy chunk

In [ ]:
null_values = combined_df_cleaned[combined_df_cleaned['policy_chunk'].isna()]

print(f"Number of rows with NaN in 'policy_chunk': {null_values.shape[0]}")
print(null_values[['doc_id', 'url', 'policy_chunk']])

# Check for rows where 'policy_chunk' is an empty string
empty_chunks = combined_df_cleaned[combined_df_cleaned['policy_chunk'].str.strip() == '']

print(f"Number of rows with empty 'policy_chunk': {empty_chunks.shape[0]}")
print(empty_chunks[['doc_id', 'url', 'policy_chunk']])


Number of rows with NaN in 'policy_chunk': 0
Empty DataFrame
Columns: [doc_id, url, policy_chunk]
Index: []
Number of rows with empty 'policy_chunk': 87
       doc_id                                                url policy_chunk
4        4890  http://minecraft.gamepedia.com/Minecraft_Wiki:...             
193     14173  https://www.ted.com/about/our-organization/our...             
309      5751        http://www.uptodate.com/home/privacy-policy             
374       750               http://www.adweek.com/privacy-policy             
492      2443  http://mlb.mlb.com/mlb/help/mlb_help_about_pri...             
...       ...                                                ...          ...
21983   13127                       https://www.usa.gov/policies             
22128    8636  http://www.gamestop.com/gs/help/PrivacyPolicy....             
22363    5801            http://dictionary.reference.com/privacy             
23017    5124   http://store.steampowered.com/privacy_agreement/   

### 5. Drop the rows where policy chunk is empty

In [ ]:
combined_df_cleaned = combined_df_cleaned[combined_df_cleaned['policy_chunk'].str.strip() != '']

print(f"Number of rows after dropping empty 'policy_chunk': {combined_df_cleaned.shape[0]}")


Number of rows after dropping empty 'policy_chunk': 10810


### 6. add another columns

In [ ]:
def extract_subcategories(segment_details):
    try:
        details_dict = json.loads(segment_details)

        return list(details_dict.keys())
    except (json.JSONDecodeError, TypeError):
        return []

combined_df_cleaned['sub_labels'] = combined_df_cleaned['segment_details'].apply(extract_subcategories)

combined_df_cleaned[['doc_id', 'segment_details', 'sub_labels', 'policy_category']].head()

print(combined_df_cleaned['sub_labels'].isna().sum())

0


### 7. Recheck the policy category distributions once again

In [ ]:
category_count = combined_df_cleaned['policy_category'].value_counts()

fig = px.bar(category_count,
             x=category_count.index,
             y=category_count.values,
             labels={'x': 'Policy Category', 'y': 'Count'},
             title='Distribution of Policy Categories',
             width=800,
             height=600)

fig.update_layout(title={'text': 'Distribution of Policy Categories', 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},xaxis_title='Policy Category' )

fig.show()


### 8. check for sub label column distribution too

In [ ]:
sub_label_counts = pd.Series([label for sublist in combined_df_cleaned['sub_labels'] for label in sublist]).value_counts()

sub_label_counts_df = sub_label_counts.reset_index()
sub_label_counts_df.columns = ['Sub Label', 'Frequency']

fig = px.bar(sub_label_counts_df,
             x='Sub Label',
             y='Frequency',
             title='Distribution of Sub Labels',
             width=800,
             height=600)

fig.update_layout(title={'text': 'Distribution of Sub Labels', 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},
                  xaxis_title='Sub Label',
                  yaxis_title='Frequency')

fig.show()


### 7. Train, test and val split and Augmentation

In [ ]:
!pip install transformers


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name_en_to_fr = 'Helsinki-NLP/opus-mt-en-fr'
tokenizer_en_to_fr = MarianTokenizer.from_pretrained(model_name_en_to_fr)
model_en_to_fr = MarianMTModel.from_pretrained(model_name_en_to_fr)

model_name_fr_to_en = 'Helsinki-NLP/opus-mt-fr-en'
tokenizer_fr_to_en = MarianTokenizer.from_pretrained(model_name_fr_to_en)
model_fr_to_en = MarianMTModel.from_pretrained(model_name_fr_to_en)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

...

target.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def translate(text, tokenizer, model):
    tokenized_text = tokenizer.prepare_seq2seq_batch([text], return_tensors='pt')

    translated = model.generate(**tokenized_text)

    translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)[0]

    return translated_text


In [ ]:
category_counts = combined_df_cleaned['policy_category'].value_counts()

threshold = 500

underrepresented_categories = category_counts[category_counts < threshold].index.tolist()

print(f"Underrepresented Categories: {underrepresented_categories}")


Underrepresented Categories: ['User Access, Edit and Deletion', 'Policy Change', 'Data Retention', 'Do Not Track']


In [ ]:
underrepresented_df = combined_df_cleaned[combined_df_cleaned['policy_category'].isin(underrepresented_categories)]

underrepresented_chunks = underrepresented_df['policy_chunk'].tolist()

for chunk in underrepresented_chunks[:5]:
    print(chunk)


By visiting the Site(s) or using any of the Curse Features, you consent to Curse' collection, use, storage, deletion and disclosure of information relating to you as set forth in this Privacy Policy. By visiting the Site(s) or using any of the Curse Features, you consent to Curse' collection, use, storage, deletion and disclosure of information relating to you as set forth in this Privacy Policy. By visiting the Site(s) or using any of the Curse Features, you consent to Curse' collection, use, storage, deletion and disclosure of information relating to you as set forth in this Privacy Policy.
You may update or delete your personal information or modify your account preferences for the Curse Features by accessing the "My Account" area of the Curse Features or sending us an email to support@curse.com requesting us to update or delete your personal information. r account preferences You may update or delete your personal information or modify your account preferences for the Curse Feature

In [ ]:
!pip install tqdm


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(combined_df_cleaned, test_size=0.2, stratify=combined_df_cleaned['policy_category'], random_state=42)

train_df, val_df = train_test_split(train_df, test_size=0.25, stratify=train_df['policy_category'], random_state=42)  # 0.25 of 0.8 is 0.2, so val is 20% of total data

print(f"Train set size: {train_df.shape}")
print(f"Validation set size: {val_df.shape}")
print(f"Test set size: {test_df.shape}")

train_distribution = train_df['policy_category'].value_counts(normalize=True)
val_distribution = val_df['policy_category'].value_counts(normalize=True)
test_distribution = test_df['policy_category'].value_counts(normalize=True)

print("\nTrain distribution:\n", train_distribution)
print("\nValidation distribution:\n", val_distribution)
print("\nTest distribution:\n", test_distribution)


Train set size: (6486, 11)
Validation set size: (2162, 11)
Test set size: (2162, 11)

Train distribution:
 policy_category
First Party Collection/Use              0.304348
Other                                   0.229571
Third Party Sharing/Collection          0.174992
User Choice/Control                     0.071230
Data Security                           0.069072
International and Specific Audiences    0.055504
User Access, Edit and Deletion          0.037003
Policy Change                           0.027906
Data Retention                          0.024206
Do Not Track                            0.006167
Name: proportion, dtype: float64

Validation distribution:
 policy_category
First Party Collection/Use              0.304348
Other                                   0.229880
Third Party Sharing/Collection          0.174838
User Choice/Control                     0.071230
Data Security                           0.068918
International and Specific Audiences    0.055967
User Access, Edit

### 8.Text Augmentation

The augmentation process involves using back-translation to generate new data for underrepresented categories in the dataset. The method works as follows:

Translation Setup: Two MarianMT models from the Helsinki-NLP library are used. The first model (opus-mt-en-fr) translates English text to French, and the second model (opus-mt-fr-en) translates French back to English.

Data Selection: Underrepresented classes in the dataset are identified by checking their frequency. Policy chunks from these categories are extracted for augmentation.

Back-Translation: Each policy chunk from the underrepresented categories is passed through the English-to-French translation model. The resulting French translation is then passed through the French-to-English model to generate a back-translated version. This method introduces minor variations in sentence structure while preserving the original meaning.

Augmented Sentences: The back-translated sentences are often slightly different from the original ones due to the nature of translation, leading to syntactic variations that help in creating more diverse training data. This augmentation improves the balance in the dataset by increasing the size of the underrepresented classes without manual annotation.

Data Augmentation: The back-translated policy chunks are combined with the original dataset, expanding the training set size and addressing class imbalance. As a result, the underrepresented categories get more samples, improving the model's ability to learn from them.

In [ ]:
import time
from tqdm import tqdm

threshold = 0.1

underrepresented_categories = train_df['policy_category'].value_counts(normalize=True)
underrepresented_categories = underrepresented_categories[underrepresented_categories < threshold].index.tolist()

underrepresented_chunks = train_df[train_df['policy_category'].isin(underrepresented_categories)]['policy_chunk'].tolist()

category_counts = train_df['policy_category'].value_counts()
augmentation_factors = {category: int((category_counts.max() - count) / count) for category, count in category_counts.items() if category in underrepresented_categories}

augmented_chunks = []
start_time = time.time()

for category in underrepresented_categories:
    category_chunks = train_df[train_df['policy_category'] == category]['policy_chunk'].tolist()

    for _ in range(augmentation_factors[category]):
        for chunk in tqdm(category_chunks, desc=f"Back-Translating for {category}", unit="chunk"):
            french_translation = translate(chunk, tokenizer_en_to_fr, model_en_to_fr)

            back_translation = translate(french_translation, tokenizer_fr_to_en, model_fr_to_en)

            augmented_chunks.append({
                'policy_chunk': back_translation,
                'policy_category': category
            })

end_time = time.time()

print(f"Augmentation completed in {end_time - start_time:.2f} seconds.")

augmented_df = pd.DataFrame(augmented_chunks)
train_df_augmented = pd.concat([train_df, augmented_df], ignore_index=True)

print(f"Original train set size: {train_df.shape}")
print(f"New train set size after augmentation: {train_df_augmented.shape}")


Back-Translating for User Choice/Control:   0%|          | 0/462 [00:00<?, ?chunk/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4252: FutureWarning:


`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.


Back-Translating for Data Security: 100%|██████████| 448/448 [12:11<00:00,  1.63s/chunk]
Bac

Augmentation completed in 19460.06 seconds.
Original train set size: (6486, 11)
New train set size after augmentation: (17612, 11)


In [ ]:
train_df_augmented.to_csv(folder_path + 'train_df_augmented.csv')

In [ ]:
path = '/content/drive/MyDrive/compliance/OPP-115/'

In [ ]:
train_aug_df = pd.read_csv(path + 'train_df_augmented.csv')
train_aug_df.head(2)

,Unnamed: 0,doc_id,test_category,segment_number,unique_policy_identifier,label,policy_category,segment_details,policy_date,url,policy_chunk,sub_labels
0,0,23954.0,test_category_labeling_highlight_upitt,123.0,3708.0,42.0,Other,"{""Other Type"": {""endIndexInSegment"": 276, ""sta...",5/29/15,https://www.allstate.com/about/privacy-stateme...,Please send requests to: Allstate Insurance Co...,['Other Type']
1,1,17925.0,test_category_labeling_highlight_fordham_ccccc,117.0,3849.0,15.0,User Choice/Control,"{""Choice Type"": {""selectedText"": ""changing the...",5/11/09,http://www.pbs.org/about/policies/privacy-policy/,changing the settings in your web browser Not ...,"['Choice Type', 'Choice Scope', 'User Type', '..."


In [ ]:
augmented_rows = train_aug_df[~train_aug_df.index.isin(train_df.index)]

# Display the augmented rows
print("Augmented Data:")
augmented_rows.head(2)  # Display first few augmented rows


Augmented Data:


,Unnamed: 0,doc_id,test_category,segment_number,unique_policy_identifier,label,policy_category,segment_details,policy_date,url,policy_chunk,sub_labels
1,1,17925.0,test_category_labeling_highlight_fordham_ccccc,117.0,3849.0,15.0,User Choice/Control,"{""Choice Type"": {""selectedText"": ""changing the...",5/11/09,http://www.pbs.org/about/policies/privacy-policy/,changing the settings in your web browser Not ...,"['Choice Type', 'Choice Scope', 'User Type', '..."
3,3,6010.0,test_category_labeling_highlight,88.0,3723.0,8.0,Other,"{""Other Type"": {""endIndexInSegment"": 409, ""sta...",10/1/13,http://www.rockstargames.com/privacy,Please note that this Privacy Policy applies o...,['Other Type']


In [ ]:
category_count = train_aug_df['policy_category'].value_counts()

fig = px.bar(category_count,
             x=category_count.index,
             y=category_count.values,
             labels={'x': 'Policy Category', 'y': 'Count'},
             title='Distribution of Policy Categories',
             width=800,
             height=600)

fig.update_layout(title={'text': 'Distribution of Policy Categories', 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'},xaxis_title='Policy Category' )

fig.show()


In [ ]:
test_df.to_csv(path + 'test_df.csv')
val_df.to_csv(path + 'val_df.csv')
train_aug_df.to_csv(path + 'train_aug_df.csv')